In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls
from plotly import tools
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='rrazaghi', api_key='xnbxzag0vc')

from sklearn.preprocessing import StandardScaler
%matplotlib notebook


# Loading the Dataset

In [2]:
df_good1 = pd.read_csv('./goodfit1.csv')
df_good2 = pd.read_csv('./goodfit2.csv')
df_good3 = pd.read_csv('./goodfit3.csv')
df_poor1 = pd.read_csv('./poorfit1.csv')
df_poor2 = pd.read_csv('./poorfit2.csv')
df_poor3 = pd.read_csv('./poorfit3.csv')


In [3]:
df_good3.tail()

,Time,3R Gastroc Muscle,3R Distal Tibia,3R Tibial Tubercle,3R Fibular Head,3L Gastroc Muscle,3L Distal Tibia,3L Tibial Tubercle,3L Fibular Head,Class,Time (min),Activity,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
8759,8759,33.10,31.94,30.51,28.56,29.87,32.36,32.40,32.54,sit rest with px simulator doffed,NaN,NaN,NaN,NaN,NaN,NaN
8760,8760,33.11,31.94,30.51,28.56,29.86,32.35,32.40,32.54,sit rest with px simulator doffed,NaN,NaN,NaN,NaN,NaN,NaN
8761,8761,33.10,31.94,30.52,28.56,29.86,32.35,32.41,32.54,sit rest with px simulator doffed,NaN,NaN,NaN,NaN,NaN,NaN
8762,8762,33.10,31.94,30.52,28.56,29.86,32.36,32.42,32.54,sit rest with px simulator doffed,NaN,NaN,NaN,NaN,NaN,NaN
8763,8763,33.10,31.95,30.53,28.56,29.86,32.36,32.41,32.54,sit rest with px simulator doffed,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# extracting the dataset into a matrix containing the temperature data and an array containing the class information
X_g1 = df_good1.ix[:,1:9].values
y_g1 = df_good1.ix[:,9].values
X_g2 = df_good2.ix[:,1:9].values
y_g2 = df_good2.ix[:,9].values
X_g3 = df_good3.ix[:,1:9].values
y_g3 = df_good3.ix[:,9].values
X_p1 = df_poor1.ix[:,1:9].values
y_p1 = df_poor1.ix[:,9].values
X_p2 = df_poor2.ix[:,1:9].values
y_p2 = df_poor2.ix[:,9].values
X_p3 = df_poor3.ix[:,1:9].values
y_p3 = df_poor3.ix[:,9].values

# Standardizing

In [5]:
# Here we standardize the data points before feeding them into the PCA algorithm:
Xg1_std = StandardScaler().fit_transform(X_g1)
Xg2_std = StandardScaler().fit_transform(X_g2)
Xg3_std = StandardScaler().fit_transform(X_g3)
Xp1_std = StandardScaler().fit_transform(X_p1)
Xp2_std = StandardScaler().fit_transform(X_p2)
Xp3_std = StandardScaler().fit_transform(X_p3)

# Eigendecomposition - Computing Eigenvectors and Eigenvalues

Covariance Matrix, Eigenvectors, and Eigenvalues:

In [6]:
def eigendecomp(X_std):
    
    cov_mat = np.cov(X_std.T)
    print('Covariance matrix \n%s' %cov_mat)

    eig_vals, eig_vecs = np.linalg.eig(cov_mat)

    print('Eigenvectors \n%s' %eig_vecs)
    print('\nEigenvalues \n%s' %eig_vals)

    return eig_vals, eig_vecs
eig_vals_g1, eig_vecs_g1 = eigendecomp(Xg1_std)
eig_vals_g2, eig_vecs_g2 = eigendecomp(Xg2_std)
eig_vals_g3, eig_vecs_g3 = eigendecomp(Xg3_std)
eig_vals_p1, eig_vecs_p1 = eigendecomp(Xp1_std)
eig_vals_p2, eig_vecs_p2 = eigendecomp(Xp2_std)
eig_vals_p3, eig_vecs_p3 = eigendecomp(Xp3_std)

Covariance matrix 
[[ 1.00011461  0.92288055  0.9099399   0.46749589  0.79953475  0.88415634
   0.88860384  0.90643156]
 [ 0.92288055  1.00011461  0.93810683  0.44739292  0.88438843  0.94931778
   0.90474184  0.9154363 ]
 [ 0.9099399   0.93810683  1.00011461  0.67331851  0.86723337  0.8890719
   0.91513549  0.91903443]
 [ 0.46749589  0.44739292  0.67331851  1.00011461  0.44507371  0.35704125
   0.54316992  0.51948415]
 [ 0.79953475  0.88438843  0.86723337  0.44507371  1.00011461  0.96104961
   0.96021737  0.95669224]
 [ 0.88415634  0.94931778  0.8890719   0.35704125  0.96104961  1.00011461
   0.95745196  0.96620253]
 [ 0.88860384  0.90474184  0.91513549  0.54316992  0.96021737  0.95745196
   1.00011461  0.99595169]
 [ 0.90643156  0.9154363   0.91903443  0.51948415  0.95669224  0.96620253
   0.99595169  1.00011461]]
Eigenvectors 
[[-0.35680576  0.06776584 -0.63190817  0.5112935   0.10590913  0.4198527
   0.07932012  0.11654933]
 [-0.36725946  0.13113332 -0.33765047 -0.56206169 -0.560391

# Selecting Principal Components

The typical goal of a PCA is to reduce the dimensionality of the original feature space by projecting it onto a smaller subspace, where the eigenvectors will form the axes. However, the eigenvectors only define the directions of the new axis, since they have all the same unit length 1, which can confirmed by the following two lines of code:

In [7]:
for ev in eig_vecs_g1:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
print('Everything ok!')

Everything ok!


In [8]:
def eigensort(eig_vals, eig_vecs):
    # Make a list of (eigenvalue, eigenvector) tuples
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

    # Sort the (eigenvalue, eigenvector) tuples from high to low
    eig_pairs.sort()
    eig_pairs.reverse()

    # Visually confirm that the list is correctly sorted by decreasing eigenvalues
    print('Eigenvalues in descending order:')
    for i in eig_pairs:
        print(i[0])
    return eig_pairs
eig_pairs_g1 = eigensort(eig_vals_g1, eig_vecs_g1)
eig_pairs_g2 = eigensort(eig_vals_g2, eig_vecs_g2)
eig_pairs_g3 = eigensort(eig_vals_g3, eig_vecs_g3)
eig_pairs_p1 = eigensort(eig_vals_p1, eig_vecs_p1)
eig_pairs_p2 = eigensort(eig_vals_p2, eig_vecs_p2)
eig_pairs_p3 = eigensort(eig_vals_p3, eig_vecs_p3)


Eigenvalues in descending order:
6.80756089031
0.793233257385
0.251227036391
0.100278577595
0.0230974624518
0.0164729941817
0.0064851262427
0.00256156088798
Eigenvalues in descending order:
6.82366031578
0.57621647684
0.415855607999
0.131512019892
0.0300368020564
0.0139851154078
0.00777729089578
0.00184793471348
Eigenvalues in descending order:
6.12134491692
1.4503701102
0.249696009173
0.11770365302
0.0324222311005
0.0194618341273
0.00797207064392
0.00194210418315
Eigenvalues in descending order:
6.26905081885
1.30179750754
0.235678220125
0.127944944356
0.0439064080655
0.0118554701743
0.0066729396745
0.00408661662431
Eigenvalues in descending order:
6.65096696129
0.8119143746
0.464714579893
0.0382376503121
0.0172020276411
0.00845646484439
0.00749331502672
0.00192062865252
Eigenvalues in descending order:
5.92081297072
1.35772515649
0.493940996562
0.185510282048
0.0196648429544
0.0152346089591
0.00563620230104
0.00240398421062


# Explained Variance

After sorting the eigenpairs, the next question is "how many principal components are we going to choose for our new feature subspace?" A useful measure is the so-called "explained variance," which can be calculated from the eigenvalues. The explained variance tells us how much information (variance) can be attributed to each of the principal components.

In [9]:
def explainedVar(eig_vals):
    tot = sum(eig_vals_g1)
    var_exp = [(i / tot)*100 for i in sorted(eig_vals_g1, reverse=True)]
    return var_exp
varexp_g1 = explainedVar(eig_vals_g1); varexp_g2 = explainedVar(eig_vals_g2);varexp_g3 = explainedVar(eig_vals_g3);
varexp_p1 = explainedVar(eig_vals_p1); varexp_p2 = explainedVar(eig_vals_p2);varexp_p3 = explainedVar(eig_vals_p3);
#cum_var_exp = np.cumsum(var_exp)

trace1 = go.Bar(
        x=['PC %s' %i for i in range(1,9)],
        y=varexp_g1,
        showlegend=True)
trace2 = go.Bar(
        x=['PC %s' %i for i in range(1,9)],
        y=varexp_g2,
        showlegend=False)
trace3 = go.Bar(
        x=['PC %s' %i for i in range(1,9)],
        y=varexp_g3,
        showlegend=False)
trace4 = go.Bar(
        x=['PC %s' %i for i in range(1,9)],
        y=varexp_p1,
        showlegend=False)
trace5 = go.Bar(
        x=['PC %s' %i for i in range(1,9)],
        y=varexp_p2,
        showlegend=False)
trace6 = Bar(
        x=['PC %s' %i for i in range(1,9)],
        y=varexp_p3,
        showlegend=False)

#trace2 = Scatter(x=['PC %s' %i for i in range(1,9)], y=cum_var_exp,name='cumulative explained variance')

#data = Data([trace1, trace2])

#layout=Layout(yaxis=YAxis(title='Explained variance in percent'),title='Explained variance by different principal components for the good fit trial 1')
    
fig = tools.make_subplots(rows=3, cols=2,
                          subplot_titles=('good fit 1','poor fit 1','good fit 2', 'poor fit 2','good fit 3', 'poor fit 3'),
                          specs=[[{}, {}], [{}, {}], [{}, {}]],
                          horizontal_spacing = 0.1, vertical_spacing = 0.15)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 3, 1)
fig.append_trace(trace4, 1, 2)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 3, 2)

fig['layout']['yaxis1'].update(title='EV (%)')
fig['layout']['yaxis2'].update(title='EV (%)')
fig['layout']['yaxis3'].update(title='EV (%)')
fig['layout']['yaxis4'].update(title='EV (%)')
fig['layout']['yaxis5'].update(title='EV (%)')
fig['layout']['yaxis6'].update(title='EV (%)')
fig['layout'].update(title='Explained variance by different principal components')
py.iplot(fig)


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]



As we can see above, choosing two principal componnets with the highest eigenvalue will ensure that we save more than 95 percent of the data while reducing the dimension from nine to two.

# Projection Matrix and Projection Onto the New Feature Space

In [10]:

Y1 = Xg1_std.dot(np.hstack((eig_pairs_g1[0][1].reshape(8,1), eig_pairs_g1[1][1].reshape(8,1))))
Y2 = Xg2_std.dot(np.hstack((eig_pairs_g2[0][1].reshape(8,1), eig_pairs_g2[1][1].reshape(8,1))))
Y3 = Xg3_std.dot(np.hstack((eig_pairs_g3[0][1].reshape(8,1), eig_pairs_g3[1][1].reshape(8,1))))
Y4 = Xp1_std.dot(np.hstack((eig_pairs_p1[0][1].reshape(8,1), eig_pairs_p1[1][1].reshape(8,1))))
Y5 = Xp2_std.dot(np.hstack((eig_pairs_p2[0][1].reshape(8,1), eig_pairs_p2[1][1].reshape(8,1))))
Y6 = Xp3_std.dot(np.hstack((eig_pairs_p3[0][1].reshape(8,1), eig_pairs_p3[1][1].reshape(8,1))))
# print('Matrix W:\n', matrix_w)

Here we will visualize the PCA results first for all the activities and then for the top three activities that are suspicious to play a key role in possible classification (walking on treadmill for two intervals and sit rest with px simulator doffed):

In [11]:

fig = tools.make_subplots(rows=3, cols=2,
                           subplot_titles=('good fit 1','poor fit 1','good fit 2', 'poor fit 2','good fit 3', 'poor fit 3'),
                           specs=[[{}, {}], [{}, {}], [{}, {}]],
                           horizontal_spacing = 0.1, vertical_spacing = 0.15)
i = 0
clr = ['black','orange', 'blue','red','cyan','magenta','yellow','deeppink', 'firebrick']
for name in ('don px components ', 'walk on treadmill (2)', 'walk on treadmill (1)', 'sit rest with px simulator doffed', 'sit rest with px simulator donned (3)', 'sit rest bare limb', 'doff px components', 'sit rest with px simulator donned (2)',
              'sit rest with px simulator donned (1)'):
    
    trace1 = go.Scatter(
         x=Y1[y_g1==name,0],
         y=Y1[y_g1==name,1],
         mode='markers',
         name=name,
         marker=dict(
             size=3,
             color = clr[i],
             opacity=0.8))
    fig.append_trace(trace1, 1, 1)
    
    trace2 = go.Scatter(
         x=Y2[y_g2==name,0],
         y=Y2[y_g2==name,1],
         mode='markers',
         name=name,
         marker=dict(
             size=3,
             color = clr[i],
             opacity=0.8))
    fig.append_trace(trace2, 2, 1)
    
    trace3 = go.Scatter(
         x=Y3[y_g3==name,0],
         y=Y3[y_g3==name,1],
         mode='markers',
         name=name,
         marker=dict(
             size=3,
             color = clr[i],
             opacity=0.8))
    fig.append_trace(trace3, 3, 1)
    
    trace4 = go.Scatter(
         x=Y4[y_p1==name,0],
         y=Y4[y_p1==name,1],
         mode='markers',
         name=name,
         marker=dict(
             size=3,
             color = clr[i],
             opacity=0.8))
    fig.append_trace(trace4, 1, 2)
    
    trace5 = go.Scatter(
            x=Y5[y_p2==name,0],
        y=Y5[y_p2==name,1],
         mode='markers',
         name=name,
         marker=dict(
             size=3,
             color = clr[i]))
    fig.append_trace(trace5, 2, 2)
    
    trace6 = go.Scatter(
         x=Y6[y_p3==name,0],
         y=Y6[y_p3==name,1],
         mode='markers',
         name=name,
         marker=dict(
             size=3,
             color = clr[i],
             opacity=0.8))
    fig.append_trace(trace6, 3, 2)
    i += 1


fig['layout'].update(title='PC1 VS PC2 for all the activities')

py.iplot(fig)

#from IPython.display import Image
#Image('first.png')


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]

The draw time for this plot will be slow for clients without much RAM.


C:\Users\sword\Anaconda3\lib\site-packages\plotly\plotly\plotly.py:1443: UserWarning:

Estimated Draw Time Slow



In [12]:

fig = tools.make_subplots(rows=3, cols=2,
                          subplot_titles=('good fit 1','poor fit 1','good fit 2', 'poor fit 2','good fit 3', 'poor fit 3'),
                          specs=[[{}, {}], [{}, {}], [{}, {}]],
                          horizontal_spacing = 0.1, vertical_spacing = 0.15)
i = 0
clr = ['black','orange', 'blue','red','cyan','magenta','yellow','deeppink', 'firebrick']

for name in ('walk on treadmill (2)', 'walk on treadmill (1)', 'sit rest with px simulator doffed'):
  
    trace1 = go.Scatter(
        x=Y1[y_g1==name,0],
        y=Y1[y_g1==name,1],
        mode='markers',
        name=name,
        marker=dict(
            size=1,
            color = clr[i],
            opacity=0.8))
    fig.append_trace(trace1, 1, 1)

    trace2 = go.Scatter(
        x=Y2[y_g2==name,0],
        y=Y2[y_g2==name,1],
        mode='markers',
        name=name,
        marker=dict(
            size=1,
            color = clr[i],
            opacity=0.8))
    fig.append_trace(trace2, 2, 1)
    
    trace3 = go.Scatter(
        x=Y3[y_g3==name,0],
        y=Y3[y_g3==name,1],
        mode='markers',
        name=name,
        marker=dict(
            size=1,
            color = clr[i],
            opacity=0.8))
    fig.append_trace(trace3, 3, 1)
    
    trace4 = go.Scatter(
        x=Y4[y_p1==name,0],
        y=Y4[y_p1==name,1],
        mode='markers',
        name=name,
        marker=dict(
            size=1,
            color = clr[i],
            opacity=0.8))
    fig.append_trace(trace4, 1, 2)
    
    trace5 = go.Scatter(
        x=Y5[y_p2==name,0],
        y=Y5[y_p2==name,1],
        mode='markers',
        name=name,
        marker=dict(
            size=1,
            color = clr[i]))
    fig.append_trace(trace5, 2, 2)
    
    trace6 = go.Scatter(
        x=Y6[y_p3==name,0],
        y=Y6[y_p3==name,1],
        mode='markers',
        name=name,
        marker=dict(
            size=1,
            color = clr[i],
            opacity=0.8))
    fig.append_trace(trace6, 3, 2)
    i += 1

fig['layout'].update(title='PC1 VS PC2 for walking on treadmill and sit rest with px simulator doffed ')

py.iplot(fig)



This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]



In [13]:

fig = tools.make_subplots(rows=3, cols=2,
                          subplot_titles=('good fit 1','poor fit 1','good fit 2', 'poor fit 2','good fit 3', 'poor fit 3'),
                          specs=[[{}, {}], [{}, {}], [{}, {}]],
                          horizontal_spacing = 0.1, vertical_spacing = 0.15)
i = 0
clr = ['black','orange', 'blue','red','cyan','magenta','yellow','deeppink', 'firebrick']

for name in ('walk on treadmill (2)', 'walk on treadmill (1)', 'sit rest with px simulator doffed'):
    
    trace1 = go.Scatter(
        x=np.mean(Y1[y_g1==name,0]),
        y=np.mean(Y1[y_g1==name,1]),
        mode='markers',
        name=name,
        marker=dict(
            size=10,
            color = clr[i],
            opacity=0.8))
    fig.append_trace(trace1, 1, 1)

    trace2 = go.Scatter(
        x=np.mean(Y2[y_g2==name,0]),
        y=np.mean(Y2[y_g2==name,1]),
        mode='markers',
        name=name,
        marker=dict(
            size=10,
            color = clr[i],
            opacity=0.8))
    fig.append_trace(trace2, 2, 1)
  
    trace3 = go.Scatter(
        x=np.mean(Y3[y_g3==name,0]),
        y=np.mean(Y3[y_g3==name,1]),
        mode='markers',
        name=name,
        marker=dict(
            size=10,
            color = clr[i],
            opacity=0.8))
    fig.append_trace(trace3, 3, 1)
    
    trace4 = go.Scatter(
        x=np.mean(Y4[y_p1==name,0]),
        y=np.mean(Y4[y_p1==name,1]),
        mode='markers',
        name=name,
        marker=dict(
            size=10,
            color = clr[i],
            opacity=0.8))
    fig.append_trace(trace4, 1, 2)
    
    trace5 = go.Scatter(
        x=np.mean(Y5[y_p2==name,0]),
        y=np.mean(Y5[y_p2==name,1]),
        mode='markers',
        name=name,
        marker=dict(
            size=10,
            color = clr[i]))
    fig.append_trace(trace5, 2, 2)
   
    trace6 = go.Scatter(
        x=np.mean(Y6[y_p3==name,0]),
        y=np.mean(Y6[y_p3==name,1]),
        mode='markers',
        name=name,
        marker=dict(
            size=10,
            color = clr[i],
            opacity=0.8))
    fig.append_trace(trace6, 3, 2)
    i += 1

fig['layout'].update(title='Calculated means on PC1 VS PC2 for walking on treadmill and sit rest with px simulator doffed ')

py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]



# Fast Fourier transform

A fast Fourier transform (FFT) algorithm computes the discrete Fourier transform (DFT) of a sequence, or its inverse. Fourier analysis converts a signal from its original domain (often time or space) to a representation in the frequency domain and vice versa. An FFT rapidly computes such transformations by factorizing the DFT matrix into a product of sparse (mostly zero) factors.[1] As a result, it manages to reduce the complexity of computing the DFT.

Below, we visualize the data first over the activities with the highest noise level ( taking into account that the dataset is averaged over different location in order to ease the process of visualization), then we compare the FFT output among different trials for possible classification.

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack

y1 = np.mean(df_good1[df_good1['Class']== 'walk on treadmill (1)'].ix[:,1:9].values, axis = 1)
y2 = np.mean(df_good2[df_good2['Class']== 'walk on treadmill (1)'].ix[:,1:9].values, axis = 1)
y3 = np.mean(df_good3[df_good3['Class']== 'walk on treadmill (1)'].ix[:,1:9].values, axis = 1)
y4 = np.mean(df_poor1[df_poor1['Class']== 'walk on treadmill (1)'].ix[:,1:9].values, axis = 1)
y5 = np.mean(df_poor2[df_poor2['Class']== 'walk on treadmill (1)'].ix[:,1:9].values, axis = 1)
y6 = np.mean(df_poor3[df_poor3['Class']== 'walk on treadmill (1)'].ix[:,1:9].values, axis = 1)

x1 = df_good1[df_good1['Class']== 'walk on treadmill (1)'].ix[:,0].values
x2 = df_good2[df_good2['Class']== 'walk on treadmill (1)'].ix[:,0].values
x3 = df_good3[df_good3['Class']== 'walk on treadmill (1)'].ix[:,0].values
x4 = df_poor1[df_poor1['Class']== 'walk on treadmill (1)'].ix[:,0].values
x5 = df_poor2[df_poor2['Class']== 'walk on treadmill (1)'].ix[:,0].values
x6 = df_poor3[df_poor3['Class']== 'walk on treadmill (1)'].ix[:,0].values


def fft(y):
    Fs = 150.0;  # sampling rate
    Ts = 1.0/Fs; # sampling interval
    #t = np.arange(0,1,Ts) # time vector


    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[range(n//2)] # one side frequency range

    Y = np.fft.fft(y)/n # fft computing and normalization
    Y = Y[range(n//2)]
    return Y, frq
y1f, frq1 = fft(y1)
y2f, frq2 = fft(y2)
y3f, frq3 = fft(y3)
y4f, frq4 = fft(y4)
y5f, frq5 = fft(y5)
y6f, frq6 = fft(y6)
trace1 = go.Scatter(
        x=x1,
        y=y1)
trace2 = go.Scatter(
        x=x2,
        y=y2)
trace3 = go.Scatter(
        x=x3,
        y=y3)
trace4 = go.Scatter(
        x=x4,
        y=y4)
trace5 = go.Scatter(
        x=x5,
        y=y5)
trace6 = go.Scatter(
        x=x6,
        y=y6)
fig = tools.make_subplots(rows=3, cols=2,
                          subplot_titles=('good fit 1','poor fit 1','good fit 2', 'poor fit 2','good fit 3', 'poor fit 3'),
                          specs=[[{}, {}], [{}, {}], [{}, {}]],
                          horizontal_spacing = 0.1, vertical_spacing = 0.15)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 3, 1)
fig.append_trace(trace4, 1, 2)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 3, 2)

fig['layout'].update(title='Original averaged signal over all 8 locations in the first walking on treadmill activity')


py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]



In [15]:

%matplotlib inline
fig = plt.figure()

ax1 = fig.add_subplot(321)
ax1.plot(frq1[1:], abs(y1f)[1:], 'r-')
ax1.set_title('good fit 1')
ax2 = fig.add_subplot(322)
ax2.plot(frq4[1:], abs(y4f)[1:], 'k-')
ax2.set_title('poor fit 1')
ax3 = fig.add_subplot(323)
ax3.plot(frq2[1:], abs(y2f)[1:], 'b-')
ax3.set_title('good fit 2')
ax4 = fig.add_subplot(324)
ax4.plot(frq5[1:], abs(y5f)[1:], 'g-')
ax4.set_title('poor fit 2')                
ax5 = fig.add_subplot(325)
ax5.plot(frq3[1:], abs(y3f)[1:], 'g-')
ax5.set_title('good fit 3')
ax6 = fig.add_subplot(326)
ax6.plot(frq6[1:], abs(y6f)[1:], 'g-')
ax6.set_title('poor fit 3')
plt.tight_layout()
fig = plt.gcf()

plotly_fig = tls.mpl_to_plotly( fig )
plotly_fig['layout']['title'] = 'FFT for all trials on first walking period(Amplitude VS Freq)'
plotly_fig['layout']['margin'].update({'t':50})
py.iplot(plotly_fig)

Above analysis was also performed on the second walking period and both revealed no significant difference among the datasets. Note:(It would be a good idea to run this individually on different locations without calculating the average).